# Loading Train Data

In [1]:
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [2]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '58 tGravityAcc-energy()-Y',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '483 fBodyGyro-bandsEnergy()-1,16',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '90 tBodyAccJerk-max()-X',
 '202 tBodyAccMag-std()',
 '203 tBodyAccMag-mad()',
 '215 tGravityAccMag-std()',
 '216 tGravityAccMag-mad()',
 '266 fBodyAcc-mean()-X',
 '272 fBodyAcc-mad()-X',
 '282 fBodyAcc-energy()-X',
 '303 fBodyAcc-bandsEnergy()-1,8',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [3]:
input_shape

30

In [4]:
train_names = pd.read_csv('../../../data/features.txt', delimiter = '\n', header = None)
train_column_names = train_names.values.tolist()
train_column_names = [k for row in train_column_names for k in row]

train_data = pd.read_csv('../../../data/X_train.txt', delim_whitespace = True, header = None)
train_data.columns = train_column_names

### Single dataframe column
y_train = pd.read_csv('../../../data/y_train.txt', header = None)
y_train.columns = ['Activity']

y_train_subject = pd.read_csv('../../../data/subject_train.txt', header = None)
y_train_subject.columns = ['Subject']

X_train_1 = train_data[sub_features]
X_train_2 = train_data[act_features]
X_train_data = pd.concat([X_train_1, X_train_2], axis = 1)

X_train_data = pd.concat([X_train_data, y_train, y_train_subject], axis = 1)
X_train_data

42 tGravityAcc-mean()-Y  43 tGravityAcc-mean()-Z  \
0                   -0.140840                 0.115375   
1                   -0.141551                 0.109379   
2                   -0.142010                 0.101884   
3                   -0.143976                 0.099850   
4                   -0.148750                 0.094486   
...                       ...                      ...   
7347                -0.222004                -0.039492   
7348                -0.242054                -0.039863   
7349                -0.236950                -0.026805   
7350                -0.233230                -0.004984   
7351                -0.233292                -0.020954   

      51 tGravityAcc-max()-Y  52 tGravityAcc-max()-Z  54 tGravityAcc-min()-Y  \
0                  -0.161265                0.124660               -0.123213   
1                  -0.161343                0.122586               -0.114893   
2                  -0.163711                0.094566               -0.114893   
3                  -0.163711                0.093425               -0.121336   
4                  -0.166786                0.091682               -0.121834   
...                      ...                     ...                     ...   
7347               -0.214233               -0.016391               -0.234998   
7348               -0.231477               -0.016391               -0.234998   
7349               -0.249134                0.024684               -0.216004   
7350               -0.244267                0.024684               -0.210542   
7351               -0.240956                0.003031               -0.212149   

      55 tGravityAcc-min()-Z  56 tGravityAcc-sma()  58 tGravityAcc-energy()-Y  \
0                   0.056483             -0.375426                  -0.970905   
1                   0.102764             -0.383430                  -0.970583   
2                   0.102764             -0.401602                  -0.970368   
3                   0.095753             -0.400278                  -0.969400   
4                   0.094059             -0.400477                  -0.967051   
...                      ...                   ...                        ...   
7347               -0.071977             -0.405132                  -0.918375   
7348               -0.068919             -0.358934                  -0.902880   
7349               -0.068919             -0.377025                  -0.907561   
7350               -0.040009             -0.440050                  -0.910648   
7351               -0.047491             -0.432003                  -0.910579   

      475 fBodyGyro-bandsEnergy()-1,8  483 fBodyGyro-bandsEnergy()-1,16  ...  \
0                           -0.999454                         -0.999619  ...   
1                           -0.999856                         -0.999897  ...   
2                           -0.999954                         -0.999962  ...   
3                           -0.999931                         -0.999947  ...   
4                           -0.999926                         -0.999946  ...   
...                               ...                               ...  ...   
7347                        -0.053258                         -0.307101  ...   
7348                        -0.029411                         -0.286728  ...   
7349                         0.161404                         -0.164197  ...   
7350                         0.193585                         -0.155644  ...   
7351                        -0.129277                         -0.384693  ...   

      272 fBodyAcc-mad()-X  282 fBodyAcc-energy()-X  \
0                -0.996889                -0.999968   
1                -0.997890                -0.999991   
2                -0.994097                -0.999969   
3                -0.994547                -0.999975   
4                -0.997725                -0.999990   
...                    ...                      ...   
7347             -0.050748   

In [5]:
X_train = X_train_data[(X_train_data['Subject'].isin([1, 3, 5, 7, 8])) & (X_train_data['Activity'].isin([1, 3, 4]))].iloc[:,:-2].values
y_train = X_train_data[(X_train_data['Subject'].isin([1, 3, 5, 7, 8])) & (X_train_data['Activity'].isin([1, 3, 4]))].iloc[:,-2].values

In [6]:
for k in range(len(y_train)):
    if y_train[k] == 1:
        y_train[k] = 0
    elif y_train[k] == 3:
        y_train[k] = 1
    else:
        y_train[k] = 2

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.15, shuffle = True)

In [8]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 30),
            classifier_block(30, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            nn.Linear(10, 3)
        )
    def forward(self, x):
        return self.network(x)

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [10]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 3.3818461894989014, Final Batch Loss: 1.1199910640716553
Epoch 2, Loss: 3.360764741897583, Final Batch Loss: 1.1248975992202759
Epoch 3, Loss: 3.345532536506653, Final Batch Loss: 1.108768105506897
Epoch 4, Loss: 3.321077346801758, Final Batch Loss: 1.1103602647781372
Epoch 5, Loss: 3.304026961326599, Final Batch Loss: 1.0953086614608765
Epoch 6, Loss: 3.2789058685302734, Final Batch Loss: 1.0852421522140503
Epoch 7, Loss: 3.2608437538146973, Final Batch Loss: 1.0772026777267456
Epoch 8, Loss: 3.2395211458206177, Final Batch Loss: 1.0718973875045776
Epoch 9, Loss: 3.2058804035186768, Final Batch Loss: 1.0648943185806274
Epoch 10, Loss: 3.1618908643722534, Final Batch Loss: 1.0490577220916748
Epoch 11, Loss: 3.1340503692626953, Final Batch Loss: 1.0386773347854614
Epoch 12, Loss: 3.08295476436615, Final Batch Loss: 1.0163066387176514
Epoch 13, Loss: 3.0603578090667725, Final Batch Loss: 1.0187907218933105
Epoch 14, Loss: 3.0092798471450806, Final Batch Loss: 0.99646109342

Epoch 117, Loss: 0.25040021538734436, Final Batch Loss: 0.11111901700496674
Epoch 118, Loss: 0.2344067320227623, Final Batch Loss: 0.07544846832752228
Epoch 119, Loss: 0.2423417717218399, Final Batch Loss: 0.030393123626708984
Epoch 120, Loss: 0.21522139385342598, Final Batch Loss: 0.075160451233387
Epoch 121, Loss: 0.23867261409759521, Final Batch Loss: 0.09622912108898163
Epoch 122, Loss: 0.2704731896519661, Final Batch Loss: 0.09525503218173981
Epoch 123, Loss: 0.24924403429031372, Final Batch Loss: 0.07612251490354538
Epoch 124, Loss: 0.2517349123954773, Final Batch Loss: 0.10492121428251266
Epoch 125, Loss: 0.23464252799749374, Final Batch Loss: 0.11213649809360504
Epoch 126, Loss: 0.2842940539121628, Final Batch Loss: 0.11813196539878845
Epoch 127, Loss: 0.23642971366643906, Final Batch Loss: 0.09036292880773544
Epoch 128, Loss: 0.28207990154623985, Final Batch Loss: 0.12288022041320801
Epoch 129, Loss: 0.23341692239046097, Final Batch Loss: 0.08130935579538345
Epoch 130, Loss: 0

Epoch 242, Loss: 0.20115768164396286, Final Batch Loss: 0.051896125078201294
Epoch 243, Loss: 0.15602407790720463, Final Batch Loss: 0.07615938782691956
Epoch 244, Loss: 0.12757301330566406, Final Batch Loss: 0.04269765317440033
Epoch 245, Loss: 0.17528080567717552, Final Batch Loss: 0.07691202312707901
Epoch 246, Loss: 0.1610003001987934, Final Batch Loss: 0.07317358255386353
Epoch 247, Loss: 0.12936552055180073, Final Batch Loss: 0.05861755833029747
Epoch 248, Loss: 0.14543847739696503, Final Batch Loss: 0.03927426040172577
Epoch 249, Loss: 0.1412695050239563, Final Batch Loss: 0.04589027166366577
Epoch 250, Loss: 0.1597229279577732, Final Batch Loss: 0.06416907906532288
Epoch 251, Loss: 0.15381716191768646, Final Batch Loss: 0.05352433770895004
Epoch 252, Loss: 0.15877271071076393, Final Batch Loss: 0.06000500172376633
Epoch 253, Loss: 0.1517645362764597, Final Batch Loss: 0.01764580048620701
Epoch 254, Loss: 0.13566698506474495, Final Batch Loss: 0.09244856238365173
Epoch 255, Loss

Epoch 357, Loss: 0.06246847286820412, Final Batch Loss: 0.029894530773162842
Epoch 358, Loss: 0.10733397863805294, Final Batch Loss: 0.04758930578827858
Epoch 359, Loss: 0.10496817342936993, Final Batch Loss: 0.02172013185918331
Epoch 360, Loss: 0.07544938288629055, Final Batch Loss: 0.030894987285137177
Epoch 361, Loss: 0.04752069711685181, Final Batch Loss: 0.014808394014835358
Epoch 362, Loss: 0.059573578648269176, Final Batch Loss: 0.008364670909941196
Epoch 363, Loss: 0.08780544623732567, Final Batch Loss: 0.04610219970345497
Epoch 364, Loss: 0.07970481179654598, Final Batch Loss: 0.020525842905044556
Epoch 365, Loss: 0.05276361992582679, Final Batch Loss: 0.005575676914304495
Epoch 366, Loss: 0.07088604010641575, Final Batch Loss: 0.023305796086788177
Epoch 367, Loss: 0.04808627860620618, Final Batch Loss: 0.006745961029082537
Epoch 368, Loss: 0.0912005640566349, Final Batch Loss: 0.04101990535855293
Epoch 369, Loss: 0.06199787091463804, Final Batch Loss: 0.025225330144166946
Epo

Epoch 475, Loss: 0.017920534126460552, Final Batch Loss: 0.004894621204584837
Epoch 476, Loss: 0.05655945232138038, Final Batch Loss: 0.040944408625364304
Epoch 477, Loss: 0.03479501698166132, Final Batch Loss: 0.019138945266604424
Epoch 478, Loss: 0.060238220263272524, Final Batch Loss: 0.016673361882567406
Epoch 479, Loss: 0.017656707670539618, Final Batch Loss: 0.005308769177645445
Epoch 480, Loss: 0.023847939912229776, Final Batch Loss: 0.0067277029156684875
Epoch 481, Loss: 0.043150078039616346, Final Batch Loss: 0.008668327704071999
Epoch 482, Loss: 0.019635091302916408, Final Batch Loss: 0.0037860574666410685
Epoch 483, Loss: 0.03883454203605652, Final Batch Loss: 0.019814349710941315
Epoch 484, Loss: 0.06289478065446019, Final Batch Loss: 0.03691159561276436
Epoch 485, Loss: 0.06033238861709833, Final Batch Loss: 0.031446851789951324
Epoch 486, Loss: 0.04254938894882798, Final Batch Loss: 0.011533272452652454
Epoch 487, Loss: 0.03957619518041611, Final Batch Loss: 0.01417566649

...

Epoch 4561, Loss: 0.0003183029170941154, Final Batch Loss: 4.128122327529127e-06
Epoch 4562, Loss: 0.0003183557728334563, Final Batch Loss: 0.00017873778415378183
Epoch 4563, Loss: 0.0001490422037022654, Final Batch Loss: 4.219988841214217e-05
Epoch 4564, Loss: 0.0002358189522055909, Final Batch Loss: 2.0017658243887126e-05
Epoch 4565, Loss: 0.00024916585425671656, Final Batch Loss: 0.00018929250654764473
Epoch 4566, Loss: 0.0013271396710479166, Final Batch Loss: 0.00010291762737324461
Epoch 4567, Loss: 0.00020646588018280454, Final Batch Loss: 0.00010193500202149153
Epoch 4568, Loss: 0.00020832897644140758, Final Batch Loss: 1.679013075772673e-05
Epoch 4569, Loss: 0.0028163027309346944, Final Batch Loss: 0.0004473463341128081
Epoch 4570, Loss: 0.0003007754003192531, Final Batch Loss: 1.701621113170404e-05
Epoch 4571, Loss: 0.00027683930966304615, Final Batch Loss: 1.2072439858457074e-05
Epoch 4572, Loss: 0.00035468355054035783, Final Batch Loss: 1.920722752402071e-05
Epoch 4573, Loss:

Epoch 4674, Loss: 0.002226391836302355, Final Batch Loss: 0.00111620023380965
Epoch 4675, Loss: 0.00034923615930892993, Final Batch Loss: 1.977724787138868e-05
Epoch 4676, Loss: 0.0004329071825850406, Final Batch Loss: 1.0520628165977541e-05
Epoch 4677, Loss: 0.0021057477169961203, Final Batch Loss: 3.903092510881834e-05
Epoch 4678, Loss: 0.0005899128445889801, Final Batch Loss: 0.00027353924815542996
Epoch 4679, Loss: 0.0006038811116013676, Final Batch Loss: 0.00022372904641088098
Epoch 4680, Loss: 0.0007940700634208042, Final Batch Loss: 5.1626662752823904e-05
Epoch 4681, Loss: 0.00046960580948507413, Final Batch Loss: 4.9927701184060425e-05
Epoch 4682, Loss: 0.0004850999794143718, Final Batch Loss: 3.174114317516796e-05
Epoch 4683, Loss: 0.0003397623586351983, Final Batch Loss: 6.090482202125713e-05
Epoch 4684, Loss: 0.0001752715488692047, Final Batch Loss: 6.164949809317477e-06
Epoch 4685, Loss: 0.000596554578805808, Final Batch Loss: 0.00022262831043917686
Epoch 4686, Loss: 0.0005

Epoch 4774, Loss: 0.004544177238130942, Final Batch Loss: 7.831119000911713e-05
Epoch 4775, Loss: 0.0003983257192885503, Final Batch Loss: 0.00011879115481860936
Epoch 4776, Loss: 0.0006412799484678544, Final Batch Loss: 0.00037207829882390797
Epoch 4777, Loss: 0.0006404415762517601, Final Batch Loss: 0.00022474907746072859
Epoch 4778, Loss: 0.028213199380843434, Final Batch Loss: 0.027795016765594482
Epoch 4779, Loss: 0.00045106645848136395, Final Batch Loss: 1.8458347767591476e-05
Epoch 4780, Loss: 0.00016216370568145066, Final Batch Loss: 2.2762200387660414e-05
Epoch 4781, Loss: 0.0013574085605796427, Final Batch Loss: 0.0006376233068294823
Epoch 4782, Loss: 0.006821170638431795, Final Batch Loss: 0.0057029323652386665
Epoch 4783, Loss: 0.0002219624657300301, Final Batch Loss: 4.132100002607331e-05
Epoch 4784, Loss: 0.0028657676484726835, Final Batch Loss: 6.949942326173186e-05
Epoch 4785, Loss: 0.0006605923244933365, Final Batch Loss: 0.00015887897461652756
Epoch 4786, Loss: 0.0002

Epoch 4885, Loss: 0.0003979267494287342, Final Batch Loss: 4.05868049710989e-05
Epoch 4886, Loss: 0.000493622625072021, Final Batch Loss: 4.886401438852772e-05
Epoch 4887, Loss: 0.00024934621251304634, Final Batch Loss: 3.4229382436024025e-05
Epoch 4888, Loss: 0.0003038380054931622, Final Batch Loss: 5.5696353228995577e-05
Epoch 4889, Loss: 0.003228513272915734, Final Batch Loss: 0.003089768346399069
Epoch 4890, Loss: 0.00024004638544283807, Final Batch Loss: 7.032167923171073e-05
Epoch 4891, Loss: 0.00018975671264342964, Final Batch Loss: 6.33976305834949e-05
Epoch 4892, Loss: 0.0012136803543398855, Final Batch Loss: 4.5654360292246565e-06
Epoch 4893, Loss: 0.00229896730161272, Final Batch Loss: 0.00011115866072941571
Epoch 4894, Loss: 0.00015905759391898755, Final Batch Loss: 2.1553782062255777e-05
Epoch 4895, Loss: 0.0024989431985886768, Final Batch Loss: 0.0016141294036060572
Epoch 4896, Loss: 0.00014668733638245612, Final Batch Loss: 7.911566353868693e-05
Epoch 4897, Loss: 0.00025

Epoch 4994, Loss: 0.00019834755676129134, Final Batch Loss: 3.659012872958556e-05
Epoch 4995, Loss: 0.00019573155259422492, Final Batch Loss: 4.9530815886100754e-05
Epoch 4996, Loss: 0.0002720732391026104, Final Batch Loss: 1.9299690393381752e-05
Epoch 4997, Loss: 0.00010085843041451881, Final Batch Loss: 2.018439772655256e-06
Epoch 4998, Loss: 0.00018138886116503272, Final Batch Loss: 9.330959073849954e-06
Epoch 4999, Loss: 0.0003606430400395766, Final Batch Loss: 1.576160138938576e-05
Epoch 5000, Loss: 0.00039702176218270324, Final Batch Loss: 3.1234099878929555e-05


In [11]:
softmax = nn.Softmax(dim = 1)
model.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 3))

[[44  0  2]
 [ 0 34  0]
 [ 0  0 38]]
              precision    recall  f1-score   support

           0      1.000     0.957     0.978        46
           1      1.000     1.000     1.000        34
           2      0.950     1.000     0.974        38

    accuracy                          0.983       118
   macro avg      0.983     0.986     0.984       118
weighted avg      0.984     0.983     0.983       118



In [12]:
torch.save(model.state_dict(), f'../../../saved_models/UCI 3 Label 5 Subject Classifier Ablation')